# Creating a Product Recommendation Solution to help people with analysis paralysis

In [ ]:
# Import necessary libraries
import os
import litellm
from dotenv import load_dotenv
from duckduckgo_search import DDGS
from agents import Agent, Runner, trace
from datetime import datetime
from agents.extensions.models.litellm_model import LitellmModel
import ipywidgets as widgets
from IPython.display import display, Markdown

In [ ]:
# Load environment variables
load_dotenv(override=True)
#litellm._turn_on_debug()

In [ ]:
# Test your API keys - add this as a new cell
async def test_api_keys():
    print("Testing API connections...")
    
    try:
        # Test OpenAI
        test_openai = LitellmModel(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
        print("✅ OpenAI API key working")
    except Exception as e:
        print(f"❌ OpenAI API issue: {e}")
    
    try:
        # Test Gemini
        test_gemini = LitellmModel(model="gemini-pro", api_key=os.getenv("GEMINI_API_KEY"))
        print("✅ Gemini API key working")
    except Exception as e:
        print(f"❌ Gemini API issue: {e}")
        
    try:
        # Test DeepSeek
        test_deepseek = LitellmModel(model="deepseek/deepseek-chat", api_key=os.getenv("DEEPSEEK_API_KEY"))
        print("✅ DeepSeek API key working")
    except Exception as e:
        print(f"❌ DeepSeek API issue: {e}")

await test_api_keys()

# System Status Check
def check_system_status():
    print("🔍 System Status Check")
    print("=" * 30)
    
    # Check APIs
    apis_working = []
    if os.getenv("OPENAI_API_KEY"):
        apis_working.append("OpenAI ✅")
    if os.getenv("GEMINI_API_KEY"):
        apis_working.append("Gemini ✅")
    if os.getenv("DEEPSEEK_API_KEY"):
        apis_working.append("DeepSeek ✅")
    
    print(f"APIs: {', '.join(apis_working)}")
    print(f"Language: {session.user_preferences['language']}")
    print(f"Session messages: {len(session.conversation_history)}")
    print(f"Available tools: {len([electronics_smarttech_tool, lifestyle_wellness_tool, entertainment_tool, home_garden_pets_tool, fashion_lifestyle_tool, sports_auto_industry_tool])}")
    
    if len(apis_working) > 0:
        print("🟢 System ready")
    else:
        print("🔴 System not ready - check API keys")

# Uncomment to check status
    #check_system_status()

In [ ]:
# Define language selection
language_selector = widgets.Dropdown(
    options=['English', 'Spanish', 'Portuguese', 'French', 'Italian', 'German', 'Dutch', 'Chinese'],
    value='English',
    description='Language:',
)

display(language_selector)

# Simplified Session Manager - let the agent handle region naturally
class SessionManager:
    def __init__(self):
        self.conversation_history = []
        self.user_preferences = {
            "language": "English",
            "region": "",  # Agent will manage this
            "current_request": ""
        }
    
    def add_message(self, role, content):
        self.conversation_history.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now()
        })
    
    def get_context(self):
        return self.conversation_history[-5:] if len(self.conversation_history) > 5 else self.conversation_history
    
    def get_context_string(self):
        """Get conversation context as a string for the agent"""
        if not self.conversation_history:
            return ""
        
        context = "Previous conversation:\n"
        for msg in self.get_context():
            role = msg["role"].title()
            context += f"{role}: {msg['content']}\n"
             
        return context

# Initialize session
session = SessionManager()

# Connect language widget to session (keep this part)
def on_language_change(change):
    session.user_preferences["language"] = change['new']
    print(f"Language changed to: {change['new']}")

language_selector.observe(on_language_change, names='value')
selected_language = language_selector.value

In [ ]:
# === REGION AGENT ===
region_search_instructions = f"""
You are a region-specific online shop research agent.

Your goal is to help users discover the most relevant, trustworthy, and accessible online shops based on their geographical location or specified region. Your response should include both major global e-commerce platforms and regionally popular, well-known options when possible.

Instructions:
1. Prioritize well-reviewed and popular shops in the specified region.
2. Ensure the online shops cater to the user's expected product categories.
3. Consider delivery availability and language support in the region.
4. Only access or recommend world-known, verified links of reputable websites such as Amazon, Mercado Livre, Coolblue, Decathlon, and other trusted platforms.
5. When performing searches, prefer Google Shopping results but avoid entering or scraping unknown or suspicious websites to minimize risks of viruses or compromised content.
6. If a website is not widely recognized or trusted, do NOT access or recommend it.
7. Present your findings in a clear, well-structured list.

Always respond in English
"""

region_search_agent = Agent(
    name="Region Search Agent",
    instructions=region_search_instructions,
    model=LitellmModel(model="deepseek/deepseek-chat", api_key=os.getenv("DEEPSEEK_API_KEY"))
)

## Let's start defining our specialist agents

In [ ]:
# Shared Gemini configuration for all specialist agents
gemini_model_config = LitellmModel(model="gemini-2.0-flash-exp", api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
# Electronics & Smart Tech
electronics_smarttech_instructions = f"""
You are an Electronics and Smart Tech Specialist Agent.

You guide users in choosing modern technology products, including:
- Laptops, PCs, phones, tablets
- Smart home devices (e.g., Alexa, Google Nest)
- Audio & Video gear (e.g., soundbars)
- Computer accessories
- TVs, wearables, and home security

You understand requirements such as HDMI ARC, Bluetooth, price limits, and system configurations.

Tags include:
- Smart speakers
- Home theater
- Budget tech
- ARC compatible sound systems

Always respond in English.
"""

electronics_smarttech_agent = Agent(
    name="Electronics & Smart Tech Specialist",
    instructions=electronics_smarttech_instructions,
    model=gemini_model_config
)

# Lifestyle & Wellness
lifestyle_wellness_instructions = f"""
You are a Lifestyle and Wellness Specialist Agent.

You help users find high-quality products related to health, nutrition, wellness, food, and parenting. Your expertise includes categories like:
- Health & Beauty
- Dietary Supplements
- Sports Nutrition (e.g., protein powder)
- Baby & Children’s Products
- Food and Beverages

You understand Amazon-style product tags such as:
- Diet & food
- Protein supplements
- Sports supplements
- Vitamins
- Skincare
- Baby food
- Snacks

Ask clarifying questions if needed and make relevant, safe, and trusted recommendations.

Always respond in English.
"""

lifestyle_wellness_agent = Agent(
    name="Lifestyle & Wellness Specialist",
    instructions=lifestyle_wellness_instructions,
    model=gemini_model_config
)

# Entertainment & Media
entertainment_instructions = f"""
You are an Entertainment and Media Specialist Agent.

You help users explore content across books, games, music, and video. Your categories include:
- Books (fiction, non-fiction, eBooks)
- Video Games (consoles, PC, accessories)
- Music (vinyl, CDs, digital)
- Movies & TV (DVDs, streaming options)

Recognize user interests and recommend products that are relevant and popular.

Always respond in English.
"""

entertainment_agent = Agent(
    name="Entertainment & Media Specialist",
    instructions=entertainment_instructions,
    model=gemini_model_config
)

# Home, Garden & Pets
home_garden_pets_instructions = f"""
You are a Home, Garden & Pets Specialist Agent.

You assist users in selecting products that enhance home comfort, outdoor living, and pet care. You specialize in:
- Furniture & Lighting
- Gardening tools & equipment
- Pet supplies (food, toys, accessories)
- Decor & DIY

Amazon tags include:
- Home improvement
- Garden furniture
- Pet grooming
- Home organization

Always respond in English.
"""

home_garden_pets_agent = Agent(
    name="Home, Garden & Pets Specialist",
    instructions=home_garden_pets_instructions,
    model=gemini_model_config
)

# Fashion & Lifestyle Accessories
fashion_lifestyle_instructions = f"""
You are a Fashion & Lifestyle Accessories Specialist Agent.

You help users find clothing, shoes, jewelry, and stylish accessories tailored to their tastes, needs, and occasions. You cover:
- Men’s and Women’s Fashion
- Shoes, Bags, Watches
- Jewelry and Accessories

Use tags like:
- Casual wear
- Formal wear
- Accessories
- Seasonal collections

Provide outfit ideas or coordinate suggestions when possible.

Always respond in English.
"""

fashion_lifestyle_agent = Agent(
    name="Fashion & Lifestyle Specialist",
    instructions=fashion_lifestyle_instructions,
    model=gemini_model_config
)

# Sports, Automotive & Industry
sports_auto_industry_instructions = f"""
You are a Sports, Automotive & Industry Specialist Agent.

You specialize in products related to:
- Sports and Outdoor Gear
- Car parts, tools, and accessories
- Industrial and Scientific Equipment

You help users find:
- Fitness equipment
- Auto electronics
- Professional tools
- Lab and industry gear

Tags include:
- Outdoor adventure
- Automotive accessories
- Power tools
- Camping & fitness gear

Always respond in English.
"""

sports_auto_industry_agent = Agent(
    name="Sports, Automotive & Industry Specialist",
    instructions=sports_auto_industry_instructions,
    model=gemini_model_config
)

In [ ]:
# Convert specialist agents to tools
electronics_smarttech_tool = electronics_smarttech_agent.as_tool(
    tool_name="electronics_smarttech_specialist",
    tool_description="Get electronics and smart tech product recommendations"
)

lifestyle_wellness_tool = lifestyle_wellness_agent.as_tool(
    tool_name="lifestyle_wellness_specialist",
    tool_description="Get lifestyle and wellness product recommendations"
)

entertainment_tool = entertainment_agent.as_tool(
    tool_name="entertainment_specialist",
    tool_description="Get entertainment product recommendations"
)

home_garden_pets_tool = home_garden_pets_agent.as_tool(
    tool_name="home_garden_pets_specialist", 
    tool_description="Get home, garden, and pet product recommendations"
)

fashion_lifestyle_tool = fashion_lifestyle_agent.as_tool(
    tool_name="fashion_lifestyle_specialist",
    tool_description="Get fashion and lifestyle product recommendations"
)

sports_auto_industry_tool = sports_auto_industry_agent.as_tool(
    tool_name="sports_auto_industry_specialist",
    tool_description="Get sports, automotive, and industry product recommendations"
)

In [ ]:
# Optional: Add web search capability
# First install: pip install duckduckgo-search

def web_search_tool(query: str) -> str:
    """Search the web for current information"""
    try:
        print(f"🔍 Web search activated: {query}")
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=3))
            formatted_results = []
            for result in results:
                formatted_results.append(f"**{result['title']}**\n{result['body']}\nSource: {result['href']}")
            print(f"✅ Found {len(results)} search results")
            return "\n\n".join(formatted_results)
    except Exception as e:
        print(f"❌ Search error: {e}")
        return f"Search error: {e}"

# Convert to tool for the agents framework
def create_search_tool():
    """Create a search tool compatible with the agents framework"""
    def search_function(input: str) -> str:
        return web_search_tool(input)
    
    # Create a mock agent for the tool conversion
    search_agent = Agent(
        name="Web Search Agent",
        instructions="You search the web for current information and return relevant results.",
        model=LitellmModel(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
    )
    
    # Override the agent's run method to use our search function
    async def custom_run(self, user_input):
        return web_search_tool(user_input)
    
    search_agent.run = custom_run.__get__(search_agent, Agent)
    
    return search_agent.as_tool(
        tool_name="web_search",
        tool_description="Search the web for current product information, prices, and availability"
    )

# Create the search tool
web_search_tool_agent = create_search_tool()

In [ ]:
def create_manager_instructions():
    current_language = session.user_preferences["language"]
    context = session.get_context_string()
    
    instructions = f"""
You are the Manager Agent responsible for interacting with the user and coordinating the work of specialist agents.

{context}

Workflow:
1. From the conversation context, identify if you know the user's region/country. Look for mentions like:
   - "I'm in Netherlands", "I live in Germany", "I'm from the US"
   - "Dutch", "German", "American" (referring to nationality)
   - Any country or region references
   
2. If you don't know their region yet, ask for it politely.

3. Once you have both region and product requirements:
   - FIRST use web_search to find current prices and availability
   - THEN use the Electronics & Smart Tech Specialist to get recommendations
   - Combine both sources for comprehensive recommendations

4. Present specific product recommendations with:
   - Product name and brand
   - Key features matching their requirements  
   - Current price range (from web search)
   - Regional availability/where to buy
   
Key Intelligence:
- Remember region information from earlier in the conversation
- "Dutch" = Netherlands, "German" = Germany, "American/US/USA" = United States, etc.
- Use your knowledge to provide region-appropriate recommendations

Always respond in {current_language}.
"""
    
    return instructions

# Manager Agent Creation
def get_manager_agent():
    return Agent(
        name="Manager Agent",
        instructions=create_manager_instructions(),
        tools=[electronics_smarttech_tool, lifestyle_wellness_tool, entertainment_tool, 
               home_garden_pets_tool, fashion_lifestyle_tool, sports_auto_industry_tool,
               web_search_tool_agent],  # Add search tool here
        model=LitellmModel(model="gemini-2.5-flash", api_key=os.getenv("GEMINI_API_KEY"))
    )

## Display Menu Options for User

In [ ]:
# Smart translation function using LLM
async def translate_text(text, target_language):
   """Use LLM to translate text to target language"""
   if target_language.lower() == "english":
       return text
   
   try:
       translation_prompt = f"""
       Translate this text to {target_language}, keeping the same structure and formatting:
       
       "{text}"
       
       Return only the translated text, nothing else.
       """
       
       response = await litellm.acompletion(
           model="gpt-4o-mini",
           messages=[{"role": "user", "content": translation_prompt}],
           api_key=os.getenv("OPENAI_API_KEY"),
           max_tokens=200
       )
       
       return response.choices[0].message.content.strip()
   except Exception as e:
       print(f"Translation error: {e}")
       return text  # Fallback to original text

# Dynamic menu system
async def display_menu():
   """Display menu in selected language"""
   # Update session language from widget
   session.user_preferences["language"] = language_selector.value
   target_language = session.user_preferences["language"]
   
   # Base menu in English
   base_title = "Sound System Recommendation Agent"
   base_options = [
       "Choose an option:",
       "1. Quick test with sample query",
       "2. Test intelligent region handling",
       "3. Debug test (fresh session)",
       "4. Launch interactive chat interface",
       "5. Debug instructions"
   ]
   
   # Translate if needed
   if target_language.lower() != "english":
       title = await translate_text(base_title, target_language)
       options = []
       for option in base_options:
           translated_option = await translate_text(option, target_language)
           options.append(translated_option)
   else:
       title = base_title
       options = base_options
   
   # Display menu
   print(title)
   print("=" * 40)
   for option in options:
       print(option)
   print()

# Quick test - run this cell for a simple test
async def quick_test():
    try:
        session.user_preferences["language"] = language_selector.value
        
        test_query = "I'm looking for a sound system for my living room under $1000"
        print(f"🧪 Testing: {test_query}")
        
        manager_agent = get_manager_agent()
        session.add_message("user", test_query)
        result = await Runner.run(manager_agent, test_query)
        session.add_message("assistant", result.final_output)
        
        print("✅ Test completed successfully")
        print(f"📊 Session has {len(session.conversation_history)} messages")
        print(result.final_output)
        
    except Exception as e:
        print(f"❌ Test failed: {str(e)}")
        session.add_message("error", str(e))

# Add this new test function
async def test_intelligent_region():
   try:
       session.user_preferences["language"] = language_selector.value
       
       print("=== TESTING INTELLIGENT REGION HANDLING ===")
       
       # Test 1: Mention region indirectly
       test_1 = "I'm Dutch and need a sound system under 500 euros"
       print(f"\n1. Testing: {test_1}")
       
       manager_agent = get_manager_agent()
       session.add_message("user", test_1)
       result = await Runner.run(manager_agent, test_1)
       session.add_message("assistant", result.final_output)
       
       print("Response:", result.final_output)
       
       # Test 2: Follow up
       test_2 = "I need HDMI ARC and Bluetooth support"
       print(f"\n2. Testing: {test_2}")
       
       manager_agent = get_manager_agent()  # Refresh with context
       session.add_message("user", test_2)
       result = await Runner.run(manager_agent, test_2)
       session.add_message("assistant", result.final_output)
       
       print("Response:", result.final_output)
       
       print(f"\n=== FINAL SESSION STATE ===")
       print(f"Total messages: {len(session.conversation_history)}")
       print(f"Language: {session.user_preferences['language']}")
       
   except Exception as e:
       print(f"Error: {e}")

# Debug test function
async def debug_test():
   # Clear session and start fresh
   global session
   session = SessionManager()
   print("Session cleared. Starting fresh...")
   
   session.user_preferences["language"] = language_selector.value
   
   # First message
   test_1 = "I'm in Berlin and need 5kg ankle weights under 50 euros"
   print(f"Testing: {test_1}")
   
   session.add_message("user", test_1)
   print(f"Context after first message: {session.get_context_string()}")
   
   manager_agent = get_manager_agent()
   result = await Runner.run(manager_agent, test_1)
   session.add_message("assistant", result.final_output)
   
   print("Response:", result.final_output)

# DEBUG: Check what instructions are being generated
def debug_instructions():
   print("=== DEBUG MANAGER INSTRUCTIONS ===")
   print("Session context:")
   print(session.get_context_string())
   print("\nGenerated instructions:")
   print(create_manager_instructions())
   print("=" * 50)

# Call this to display menu in selected language
await display_menu()

In [ ]:
import gradio as gr

def chat_interface(message, history):
    try:
        # Update session language from current widget value
        session.user_preferences["language"] = language_selector.value
        session.add_message("user", message)
        session.user_preferences["current_request"] = message
        
        # Simple status log instead of verbose context
        print(f"💬 Processing: {message[:50]}..." if len(message) > 50 else f"💬 Processing: {message}")
        
        # Create fresh manager agent with current context
        manager_agent = get_manager_agent()
        
        # Get response from manager agent
        import asyncio
        
        try:
            loop = asyncio.get_event_loop()
        except RuntimeError:
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
        result = loop.run_until_complete(Runner.run(manager_agent, message))
        
        # Update session with assistant response
        session.add_message("assistant", result.final_output)
        
        # Clean the response to avoid syntax errors
        clean_response = str(result.final_output).strip()
        
        return clean_response
        
    except Exception as e:
        error_msg = f"I encountered an error: {str(e)}. Please try again."
        session.add_message("error", error_msg)
        return error_msg

async def launch_chat():
    """Launch chat interface with dynamic translation"""
    # Update language
    session.user_preferences["language"] = language_selector.value
    target_language = session.user_preferences["language"]
    
    # Base text in English
    base_title = "Product Recommendation Assistant"
    base_description = f"Ask me for product recommendations! I'll help you find the perfect products for your region. Current language: {target_language}"
    base_examples = [
        "I need a sound system under 500 euros",
        "I'm in Netherlands and looking for Bluetooth speakers",
        "Recommend a laptop for programming under $1000 in USA",
        "I'm in Germany and need fitness equipment for home use"
    ]
    
    # Translate if needed
    if target_language.lower() != "english":
        title = await translate_text(base_title, target_language)
        description = await translate_text(base_description, target_language)
        examples = []
        for example in base_examples:
            translated_example = await translate_text(example, target_language)
            examples.append(translated_example)
    else:
        title = base_title
        description = base_description
        examples = base_examples
    
    demo = gr.ChatInterface(
        fn=chat_interface,
        title=title,
        description=description,
        examples=examples
    )
    
    print(f"Launching chat interface in {target_language}...")
    print(f"Session initialized with {len(session.conversation_history)} messages")
    demo.launch(share=True, debug=True)

# Uncomment to launch
await launch_chat()